In [ ]:
import pandas as pd
import numpy as np
import pathlib as pl
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from scipy.stats import ks_2samp
from tqdm.notebook import tqdm
from adjustText import adjust_text

In [ ]:
import sys
sys.path.append("../../FinalCode/")
import download.download_gex as dwnl
import utils.plotting as plting

In [ ]:
colors = sns.color_palette("muted")

In [ ]:
fig_dir = pl.Path("/add/path/here/")

# Load mapping

In [ ]:
data_dir = pl.Path("/add/path/here/")

In [ ]:
mapping_roadmap = pd.read_csv(data_dir / "NIH_Epigenomics_Roadmap" / "EPIC_to_state_mapping.csv",index_col=0)

In [ ]:
union_cpgs = pd.read_csv(data_dir / "adVMP" / "union_cpgs.csv",index_col=0).values.ravel()

In [ ]:
epic_manifest = pd.read_csv(data_dir / "illumina_manifests" / "GPL21145_MethylationEPIC_15073387_v-1-0.csv.gz",skiprows=7,index_col=0)

In [ ]:
epic_manifest = epic_manifest.loc[union_cpgs]
epic_manifest = pd.concat([epic_manifest,mapping_roadmap],axis=1,join="inner")

In [ ]:
red_manifest = epic_manifest[['CHR', 'MAPINFO',
       'UCSC_RefGene_Name', 
       'UCSC_RefGene_Group', 'UCSC_CpG_Islands_Name',
       'Relation_to_UCSC_CpG_Island', '450k_Enhancer',"State"]]

In [ ]:
tssA_manifest = red_manifest[red_manifest.State.isin(["1_TssA","2_TssAFlnk","3_TxFlnk"])]
tss_unique_genes = np.unique(np.concatenate(tssA_manifest["UCSC_RefGene_Name"].dropna().str.split(";").values))

# Download data

In [ ]:
gex_path = pl.Path("/add/path/here")
path_right = gex_path  / "GSE76987_RightColonProcessed.csv"
path_right_cr = gex_path / "GSE76987_ColonCancerProcessed.csv"

In [ ]:
# preprocesses the data
right_data = dwnl.download_gex_data(path_right=path_right, path_right_cr=path_right_cr)

In [ ]:
test_genes = right_data.columns.intersection(tss_unique_genes)

In [ ]:
# order the tissue types from healthy to cancer
ordered_type = pd.Categorical(right_data.type, 
                      categories=["Healthy","NAC","Adenoma","SSL","Cancer"],
                      ordered=True)

right_data["Ordered type"] = ordered_type

ordered_idx = right_data.sort_values(by="Ordered type").index

In [ ]:
# get the PCA representation in the subset of aDVMC-related genes
pca = PCA(n_components=2)
X_PCA = pca.fit_transform(right_data.loc[:,test_genes])
X_PCA = pd.DataFrame(X_PCA, index=right_data.index, columns=["PCA1","PCA2"])
X_PCA = pd.concat([X_PCA, right_data["Ordered type"]],axis=1)
print(pca.explained_variance_ratio_)

In [ ]:
ax = sns.scatterplot(data=X_PCA, x="PCA1", y="PCA2", hue="Ordered type")
plting.transform_plot_ax(ax, legend_title="", remove_ticks=True)
ax.set_xlabel(f"PCA1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
ax.set_ylabel(f"PCA2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
ax.figure.savefig(fig_dir / "PCA_gex_tssgenes.svg", bbox_inches='tight')

In [ ]:
# get the density plots for each PC 
fig, ax = plt.subplots(1,2,figsize=(10,3))
sns.kdeplot(data=X_PCA,x="PCA1",hue="Ordered type", ax=ax[0], legend=False, common_norm=False)
sns.kdeplot(data=X_PCA,x="PCA2",hue="Ordered type", ax=ax[1], common_norm=False)
ax[0].spines[['right', 'top']].set_visible(False)
ax[0].spines[["bottom", "left"]].set_linewidth(4)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].spines[['right', 'top']].set_visible(False)
ax[1].spines[["bottom", "left"]].set_linewidth(4)
ax[1].set_xticks([])
ax[1].set_yticks([])
fig.savefig(fig_dir / "gex_density_pca_advmp_genes.svg", bbox_inches="tight")

In [ ]:
ks_2samp(X_PCA[X_PCA["Ordered type"]=="Healthy"]["PCA1"],X_PCA[X_PCA["Ordered type"]!="Healthy"]["PCA1"]),ks_2samp(X_PCA[X_PCA["Ordered type"]=="Healthy"]["PCA2"],X_PCA[X_PCA["Ordered type"]!="Healthy"]["PCA2"])

In [ ]:
pca = PCA(n_components=2)
X_PCA = pca.fit_transform(right_data.drop(["type","Ordered type"],axis=1))
X_PCA = pd.DataFrame(X_PCA, index=right_data.index, columns=["PCA1","PCA2"])
X_PCA = pd.concat([X_PCA, right_data["Ordered type"]],axis=1)
print(pca.explained_variance_ratio_)

In [ ]:
ax = sns.scatterplot(data=X_PCA, x="PCA1", y="PCA2", hue="Ordered type")
plting.transform_plot_ax(ax, legend_title="", remove_ticks=True)
ax.set_xlabel(f"PCA1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
ax.set_ylabel(f"PCA2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
ax.figure.savefig(fig_dir / "PCA_gex.svg", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,3))
sns.kdeplot(data=X_PCA,x="PCA1",hue="Ordered type", ax=ax[0], legend=False, common_norm=False)
sns.kdeplot(data=X_PCA,x="PCA2",hue="Ordered type", ax=ax[1], common_norm=False)
ax[0].spines[['right', 'top']].set_visible(False)
ax[0].spines[["bottom", "left"]].set_linewidth(4)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].spines[['right', 'top']].set_visible(False)
ax[1].spines[["bottom", "left"]].set_linewidth(4)
ax[1].set_xticks([])
ax[1].set_yticks([])
fig.savefig(fig_dir / "gex_density_pca.svg", bbox_inches="tight")

In [ ]:
ks_2samp(X_PCA[X_PCA["Ordered type"]=="Healthy"]["PCA1"],X_PCA[X_PCA["Ordered type"]!="Healthy"]["PCA1"]),ks_2samp(X_PCA[X_PCA["Ordered type"]=="Healthy"]["PCA2"],X_PCA[X_PCA["Ordered type"]!="Healthy"]["PCA2"])

# Get differential expression

In [ ]:
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
def get_p_values_cond(data: pd.DataFrame, test_genes: np.array, 
                      pheno1: pd.Series, pheno2: pd.Series) -> np.ndarray:
    p_values = []
    for g in data.columns.intersection(test_genes):
        df = data[g]
        # we use the Wilcoxon rank sum test to compute differential gene expression
        p = mannwhitneyu(df[pheno1].values.ravel(),
                         df[pheno2].values.ravel())[1]
        p_values.append(p)
    return np.array(p_values)

In [ ]:
pheno1 = (right_data["type"]=="Healthy")
pheno2 = (right_data["type"]=="NAC")
NAC_p = get_p_values_cond(data=right_data, 
                               test_genes=right_data.columns[:-2], 
                               pheno1=pheno1, pheno2=pheno2)

In [ ]:
from scipy.stats import fisher_exact

In [ ]:
# compare the number of differentially expression aDVMC-related genes compared to all possible genes
full_dgex = pd.DataFrame(NAC_p,columns=["p"],index=right_data.columns[:-2])

full_dgex = full_dgex<.05

nunion_genes = np.setdiff1d(right_data.columns[:-2], test_genes)

a = full_dgex.loc[test_genes,"p"].sum()
b = len(test_genes) - a
c = full_dgex.loc[nunion_genes,"p"].sum()
d = len(nunion_genes) - c

conting = np.array([[a,b],[c,d]])
fisher_exact(conting)

In [ ]:
conting

In [ ]:
a/(a+b), c/(c+d)

# Volcano plots

In [ ]:
goi = test_genes

def get_gex(goi: np.ndarray) -> pd.DataFrame:
    # compute the differential gene expression results for each (condition,control), where control is entirely healthy tissue ("Healthy"),
    #  and condition is any evolution along the neoplastic transformation path
    pheno1 = (right_data["type"]=="Healthy")
    pheno2 = (right_data["type"]=="NAC")
    NAC_p = get_p_values_cond(data=right_data, 
                               test_genes=goi, 
                               pheno1=pheno1, pheno2=pheno2)
    pheno2 = (right_data["type"]=="Adenoma")
    ad_p = get_p_values_cond(data=right_data, 
                               test_genes=goi, 
                               pheno1=pheno1, pheno2=pheno2)
    pheno2 = (right_data["type"]=="SSL")
    ssl_p = get_p_values_cond(data=right_data, 
                               test_genes=goi, 
                               pheno1=pheno1, pheno2=pheno2)
    pheno2 = (right_data["type"]=="Cancer")
    cancer_p = get_p_values_cond(data=right_data, 
                               test_genes=goi, 
                               pheno1=pheno1, pheno2=pheno2)

    diff_expr = pd.DataFrame(np.array([NAC_p,ad_p,ssl_p,cancer_p]),
                             index=["NAC_p","Ad_p","SSL_p","Cancer_p"],columns=goi).T
    # compute FDR correction
    for i,col in enumerate(diff_expr.columns):
        q_value = multipletests(diff_expr[col],method="fdr_bh")[1]
        diff_expr[col[:-1]+"q"] = q_value
    return diff_expr

In [ ]:
def get_volcano_plot(right_data: pd.DataFrame, 
                     pheno1_name: str, 
                     pheno2_name: str, 
                     diff_expr: pd.DataFrame,
                     goi: np.ndarray, lim_fc: float=1.5) -> plt.Figure:

    pheno1 = (right_data["type"]==pheno1_name)
    pheno2 = (right_data["type"]==pheno2_name)
    # compute the log2FC between the condition and control
    log2FC = np.log2(right_data.loc[pheno2,goi].mean()/right_data.loc[pheno1,goi].mean())
    # the volcano plot will show the p values
    volcano_df = pd.concat([log2FC,
                            diff_expr[f"{pheno2_name}_p"].apply(lambda x: -np.log10(x)),
                            diff_expr[f"{pheno2_name}_q"].apply(lambda x: -np.log10(x))],axis=1)
    volcano_df.columns = ["log2(FC)","-log10(p)","-log10(q)"]
    # indicator of if the gene passes the FDR threshold of 0.1
    volcano_df["Significant"] = volcano_df["-log10(q)"]>=1

    # annotate only significant genes with a big enough log2FC
    genes_to_annotate = volcano_df[(volcano_df["log2(FC)"].abs()>np.log2(lim_fc)) & (volcano_df["Significant"])]

    pmin = volcano_df[volcano_df["Significant"]]["-log10(p)"].min()
    fig, ax = plt.subplots(1,1)
    sns.scatterplot(data=volcano_df, 
                    y="-log10(p)", 
                    x="log2(FC)", 
                    hue="Significant", 
                    palette = {True: "red", False: "blue"},
                    ax=ax)
    ax.hlines(xmin=ax.get_xlim()[0],xmax=ax.get_xlim()[1].max(),y=pmin,color="r",linestyle='--')
    plting.transform_plot_ax(ax, legend_title="Sign. diff. expressed")

    texts = []
    for g in genes_to_annotate.index:
        x = genes_to_annotate.loc[g,"log2(FC)"]
        y = genes_to_annotate.loc[g,"-log10(p)"]
        texts.append(ax.text(x,y,g,fontsize=10))

    adjust_text(texts, only_move={'points':'y', 'texts':'y'}, arrowprops=dict(arrowstyle="-", color='r', lw=0.5))
    return fig

In [ ]:
diff_expr = get_gex(goi=goi)

In [ ]:
(diff_expr.loc[:,diff_expr.columns.str.endswith("_q")]<0.1).sum(axis=1).value_counts()

In [ ]:
sign_diff = diff_expr["NAC_q"].apply(lambda x: -np.log10(x)).sort_values(ascending=False).to_frame()

In [ ]:
fig = get_volcano_plot(right_data=right_data, pheno1_name="Healthy", pheno2_name="NAC", goi=test_genes)
fig.savefig(fig_dir / "volcano_plot_nac_vs_healthy.svg", bbox_inches="tight")

In [ ]:
fig = get_volcano_plot(right_data=right_data, pheno1_name="Healthy", pheno2_name="SSL", goi=test_genes)
fig.savefig(fig_dir / "volcano_plot_nac_vs_SSL.svg", bbox_inches="tight")

In [ ]:
fig = get_volcano_plot(right_data=right_data, pheno1_name="Healthy", pheno2_name="Cancer", goi=test_genes, lim_fc=2.5)
fig.savefig(fig_dir / "volcano_plot_nac_vs_cancer.svg", bbox_inches="tight")

In [ ]:
# which genes are significantly dysregulated in all cases?
fully_dysregulated = diff_expr[(diff_expr["NAC_q"]<0.1) & (diff_expr["SSL_q"]<0.1) & (diff_expr["Cancer_q"]<0.1)].sort_values(by="NAC_p").index.to_numpy()

In [ ]:
def get_gex_boxplot(right_data: pd.DataFrame, gene: str) -> plt.Figure:
    fig, ax = plt.subplots(1,1,figsize=(3,2))
    sns.boxplot(data=right_data, 
                x="type", y=gene, ax=ax,
                order=["Healthy","NAC","Adenoma","SSL","Cancer"], 
                     palette=palette)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment="right")
    ax.set_xlabel("")
    ax.spines[['right', 'top']].set_visible(False)
    ax.spines[["bottom", "left"]].set_linewidth(3)
    return fig

In [ ]:
genes_to_plot = fully_dysregulated
for gene in genes_to_plot:
    figure = get_gex_boxplot(right_data=right_data, gene=gene)
    figure.savefig(fig_dir / "diffgexboxplots" / f"{gene}.svg", bbox_inches="tight")